In [4]:
import gspread
gc = gspread.service_account(filename='creds/lifts-service.json')
sheet = gc.open_by_key('1XmPko5pbcw0HdvPZeL7Fw8tfX4zhvxREUDGISC54jwk')
ws = sheet.get_worksheet(0)  


In [7]:

date = ws.col_values(1)
weight_kg = ws.col_values(4)

In [69]:
import pandas as pd

df = pd.DataFrame(
    {
        "date": date[1:],
        "weight": weight_kg[1:]
    }
)

df = df.astype(
    {'weight': 'float'},
    {'date': 'datetime'})

df['date'] = pd.to_datetime(df['date'], dayfirst=True)
df = df[df['weight'] != 0]
df

,date,weight
0,2021-04-11,80.65
1,2021-04-18,80.92
2,2021-04-25,80.29
3,2021-05-02,80.65
4,2021-05-09,80.56
...,...,...
62,2022-07-31,73.94
63,2022-08-07,74.30
64,2022-08-14,74.39
65,2022-08-21,75.84


In [145]:
import altair as alt
alt.renderers.enable('html')

chart = alt.Chart(df).mark_line(point = True).encode(
    alt.X('date:T', timeUnit='yearmonthdate', sort='-x', title='date', axis= alt.Axis(
        grid=False)),
    alt.Y('weight', scale=alt.Scale(zero=False), title = 'weight (kg)') 
).properties(
    width=1500,
    height=250
)


chart

alt.Chart(...)